In [ ]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.master("local[*]").appName("SimpleJson").config("spark.jars.packages","org.postgresql:postgresql:42.7.8").getOrCreate()

spark

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

fund_schema = StructType([
    StructField("fund_id", IntegerType(), True),
    StructField("fund_name", StringType(), True),
    StructField("fund_code", StringType(), True),
    StructField("fund_description", StringType(), True),
    StructField("updated_at", LongType(), True),
    StructField("fund_price", IntegerType(), True)
])

source_schema = StructType([
    StructField("version", StringType(), True),
    StructField("connector", StringType(), True),
    StructField("name", StringType(), True),
    StructField("ts_ms", LongType(), True),
    StructField("snapshot", StringType(), True),
    StructField("db", StringType(), True),
    StructField("sequence", StringType(), True),
    StructField("schema", StringType(), True),
    StructField("table", StringType(), True),
    StructField("txId", LongType(), True),
    StructField("lsn", LongType(), True),
    StructField("xmin", StringType(), True)
])

cdc_schema = StructType([
    StructField("before", fund_schema, True),
    StructField("after", fund_schema, True),
    StructField("source", source_schema, True),
    StructField("op", StringType(), True),
    StructField("ts_ms", LongType(), True),
    StructField("transaction", StringType(), True),
    StructField("source_system", StringType(), True)
])


In [ ]:
df = spark.read.format("json").schema(cdc_schema).option("multiLine", True).load("./data/*.json")

In [ ]:
df.printSchema()

In [ ]:
df.show(truncate=False)

In [ ]:
from pyspark.sql.functions import col, when, to_timestamp

actual_df = (
    df.select(
        col("op").alias("Operation"),
        to_timestamp((col("ts_ms") / 1000)).alias("ProcessTime"),
        col("before"),
        col("after")
    )
    .withColumn(
        "Values",
        when(col("Operation") == "d", col("before"))  
        .otherwise(col("after"))                      
    )
    .drop("before", "after")
)


actual_df.show(truncate=False)


In [ ]:
from pyspark.sql.functions import current_timestamp
final_df = (
    actual_df
    .filter(
        col("Operation").isNotNull() &
        col("Values.fund_id").isNotNull()
    )
    .select(
        col("Operation"),
        col("ProcessTime"),
        col("Values.fund_id").alias("fund_id"),
        col("Values.fund_name").alias("fund_name"),
        col("Values.fund_code").alias("fund_code"),
        col("Values.fund_description").alias("fund_description"),
        to_timestamp(col("Values.updated_at") / 1_000_000).alias("updated_at"),
        col("Values.fund_price").alias("fund_price")
    ).withColumn(
        "delete_flag",
        when(col("Operation") == "d", "Y").otherwise("N")
    )
    .withColumn(
        "effective_date",
        current_timestamp()
    )
)

final_df.show(truncate=False)


In [ ]:
jdbc_url = "jdbc:postgresql://localhost:5432/finance"
jdbc_properties = {
    "user": "pguser",
    "password": "pgpassword",
    "driver": "org.postgresql.Driver"
}

In [ ]:

def write_to_postgres(batch_df, batch_id):

    print(f"\n--- Writing Batch {batch_id} ---")
    batch_df.show(truncate=False) 
    
    batch_df.write \
        .mode("append") \
        .jdbc(url=jdbc_url, table="public.fund_metadata_trail", properties=jdbc_properties)
    
    print(f"Batch {batch_id} written to Postgres.")

write_to_postgres(final_df, batch_id=0)


In [22]:
spark.stop()